In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 15.308 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 0.882 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 1.625 seconds


In [3]:
%%hive 
SELECT * FROM t0;

SELECT * FROM t0;
OK
E	["b","g","f"]	{"jjj":3,"bbb":0,"ddd":9,"ggg":8,"hhh":2}
A	["a","f","c"]	{"ccc":2,"ddd":0,"aaa":3,"hhh":9}
B	["f","e","a","c"]	{"ddd":2,"ggg":5,"ccc":6,"jjj":1}
A	["a","b"]	{"hhh":9,"iii":5,"eee":7,"bbb":1}
C	["f","g","d","a"]	{"iii":6,"ddd":5,"eee":4,"jjj":3}
A	["c","d"]	{"bbb":2,"hhh":0,"ccc":4,"fff":1,"aaa":7}
A	["g","d","a"]	{"aaa":5,"fff":8,"ddd":2,"iii":0,"jjj":7,"ccc":1}
B	["b","a"]	{"fff":3,"hhh":1,"ddd":2}
E	["d","e","a","f"]	{"eee":4,"ccc":5,"iii":9,"fff":7,"ggg":6,"bbb":0}
B	["d","b","g","f"]	{"bbb":7,"jjj":9,"fff":5,"iii":4,"ggg":2,"eee":3}
C	["d","c","f","b"]	{"hhh":6,"eee":4,"iii":0,"fff":2,"jjj":1}
C	["d","e","a","c"]	{"bbb":7,"iii":6,"ggg":9}
D	["g","e","f","b"]	{"bbb":9,"aaa":3,"ccc":6,"fff":4,"eee":2}
E	["c","f"]	{"aaa":8,"ddd":5,"jjj":1}
B	["d","b"]	{"ccc":0,"jjj":6,"fff":7,"ddd":3,"aaa":2}
D	["f","e"]	{"ccc":0,"eee":6,"bbb":9,"ddd":3}
E	["e","b","f"]	{"bbb":6,"iii":3,"hhh":5,"fff":4,"ggg":9,"ddd":2}
D	["g","a"]	{"hhh":4,"jjj":5,"ccc":9}
E	["e",

In [6]:
%%hive 
SELECT c1, size(c2), size(c3) FROM t0;

SELECT c1, size(c2), size(c3) FROM t0;
OK
E	3	5
A	3	4
B	4	4
A	2	4
C	4	4
A	2	5
A	3	6
B	2	3
E	4	6
B	4	6
C	4	5
C	4	3
D	4	5
E	2	3
B	2	5
D	2	4
E	3	6
D	2	3
E	4	3
E	2	3
E	2	3
E	3	3
D	3	3
A	3	5
E	2	6
E	3	6
A	3	3
E	3	5
A	2	5
C	4	6
A	2	5
D	2	6
E	2	4
B	3	6
B	3	5
D	2	3
B	2	5
C	4	3
E	2	3
E	3	3
Time taken: 0.192 seconds, Fetched: 40 row(s)


In [7]:
%hive_quit

In [8]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 16:08 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 15:33 /tmp/wordcount


In [9]:
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

mkdir: `/tmp/wordcount': File exists


In [10]:
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 16:08 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 15:33 /tmp/wordcount


In [11]:
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal 'data.tsv'  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/data.tsv': File exists


In [12]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 1 items
-rw-r--r--   1 root supergroup       1394 2020-02-28 15:33 /tmp/wordcount/data.tsv


In [13]:
####################################################################################################################
## Generación del script y ajuste del código

In [14]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

SELECT c1, size(c2), size(c3) FROM t0;

Writing wordcount.hql


In [15]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [16]:
### Copia de los resultados a la maquina local (vagrant)
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [17]:
!cat output/000000_0 | head

E,3,5
A,3,4
B,4,4
A,2,4
C,4,4
A,2,5
A,3,6
B,2,3
E,4,6
B,4,6
